In [1]:
import pandas as pd
import jiwer
import re
import os

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Used to convert numbers to words, e.g. 8 o'clock -> eight o'clock, 1944 -> nineteen forty four
import inflect
# Inflect is more flexible, but doesn't create ordinals as words - use num2words for that
from num2words import num2words

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tompickard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
%cd '/home/tompickard/MiniProject/Legasee-Oral-History/'

from measures import compute_measures

from evaluation import kword_prep, score_name

from evaluation import transform_baseline

/home/tompickard/MiniProject/Legasee-Oral-History


In [3]:
SYSPATH = '/H_Drive/srv/studat/cdt/team2'
TEST_TRAIN = 'train'
M_FOLDER = 'Baseline_1_Transcript'

SHARED_WORD_WEIGHT = 3
KEY_WEIGHT = 7
TRANSFORM = transform_baseline

In [27]:
# Read metadata from CSV

META_PATH = os.path.expanduser("~")+SYSPATH+'/data/legasee/metadata/'

meta_df = pd.read_csv(META_PATH+'master_metadata.csv', converters={'Priority Words': eval, 'Name Words' : eval})

# Remove Test items
meta_df = meta_df[~pd.Series(meta_df.Allocation == "Test")]

In [28]:
# Shared keywords

all_words = [x for subl in meta_df['Priority Words'] for w in subl for x in re.split('[\s/]',w)]
all_words_clean = kword_prep(all_words,transform_baseline)


wdict = {w : SHARED_WORD_WEIGHT for w in all_words_clean}

In [36]:
_started = 0

for _fn in os.scandir(os.path.expanduser("~")+SYSPATH+'/system_outputs/'+M_FOLDER):
    # Only want to process folders. Ignore any starting with .
    if _fn.is_dir() and not _fn.name.startswith('.'):
        name_df = score_name(_fn.name,
               meta_df,
               SYSPATH,
               M_FOLDER,
               TEST_TRAIN,
               KEY_WEIGHT,
               wdict,
               TRANSFORM,
              )
        
        if type(name_df) != type(None):
            if _started:
                scores_df = scores_df.append(name_df)

            else:
                _started = 1
                scores_df = name_df.copy()

scores_df

Keywords Unweighted  \
                                                         Incidence  deletions   
Baden Singleton  [(miscellaneous, 0), (navy, 19), (singleton, 1...      121.0   
Mervyn Salter    [(overlord, 0), (jw, 0), (anti, 2), (sri, 0), ...      177.0   
Ted Rogers       [(story, 1), (merchant, 18), (cargo, 4), (conv...      213.0   
Frank Wilson                             [(wilson, 1), (frank, 2)]      207.0   
Harriet Wright   [(raf, 5), (halesworth, 2), (hatston, 9), (wri...      156.0   
Peter Dunstan    [(pong, 6), (dunstan, 1), (bomb, 7), (flow, 0)...      214.0   
John Roche       [(roche, 1), (cargo, 0), (john, 1), (sri, 0), ...      228.0   

                                                                      \
                   hits insertions       mer substitutions       wer   
Baden Singleton  5148.0      819.0  0.274112        1004.0  0.309900   
Mervyn Salter    3947.0      504.0  0.244159         594.0  0.270242   
Ted Rogers       7858.0     1111.0  0.236049        1104.0  0.264632   
Frank Wilson     3871.0      927.0  0.362798        1070.0  0.428127   
Harriet Wright   5096.0      358.0  0.180048         605.0  0.191053   
Peter Dunstan    9211.0     1167.0  0.207247        1027.0  0.230387   
John Roche       7161.0     4531.0  0.489339        2103.0  0.722925   

                          Weighted (own keywords)            ...  \
                      wip                     wer       mer  ...   
Baden Singleton  0.606048                0.310329  0.275493  ...   
Mervyn Salter    0.654508                0.276372  0.253597  ...   
Ted Rogers       0.668127                0.240276  0.218287  ...   
Frank Wilson     0.496041                0.426636  0.361727  ...   
Harriet Wright   0.731784                0.205439  0.193927  ...   
Peter Dunstan    0.711736                0.232813  0.214719  ...   
John Roche       0.391623                0.650752  0.464617  ...   

                Weighted (own + shared keywords)                       \
                                   substitutions deletions insertions   
Baden Singleton                           1280.0     133.0      819.0   
Mervyn Salter                              928.0     225.0      504.0   
Ted Rogers                                1482.0     279.0     1111.0   
Frank Wilson                              1278.0     221.0      927.0   
Harriet Wright                             847.0     206.0      358.0   
Peter Dunstan                             1903.0     406.0     1167.0   
John Roche                                2963.0     288.0     4531.0   

                Keywords (own) only                                           \
                                wer       mer       wip   hits substitutions   
Baden Singleton            0.323529  0.323529  0.457612   23.0          11.0   
Mervyn Salter              0.308725  0.308725  0.494454  103.0          41.0   
Ted Rogers                 0.119741  0.119741  0.790199  272.0          31.0   
Frank Wilson               0.000000  0.000000  1.000000    3.0           0.0   
Harriet Wright             0.689655  0.689655  0.107427    9.0          17.0   
Peter Dunstan              0.240283  0.240283  0.598312  430.0         116.0   
John Roche                 0.273927  0.273927  0.539649  220.0          76.0   

                                      
                deletions insertions  
Baden Singleton       0.0        0.0  
Mervyn Salter         5.0        0.0  
Ted Rogers            6.0        0.0  
Frank Wilson          0.0        0.0  
Harriet Wright        3.0        0.0  
Peter Dunstan        20.0        0.0  
John Roche            7.0        0.0  

[7 rows x 29 columns]

In [43]:
scores_df.to_csv(os.path.expanduser("~")+SYSPATH+'/system_outputs/'+M_FOLDER+'/evaluation_scores.tsv',sep='\t')

In [46]:
wer = scores_df['Unweighted','wer'].mean()
wwer = scores_df['Weighted (own + shared keywords)','wer'].mean()
kwer = scores_df['Keywords (own) only','wer'].mean()

In [47]:
print('WER: ', wer , 'WWER: ', wwer, 'KWER: ',kwer)

WER:  0.34532361648391774 WWER:  0.3052242799832375 KWER:  0.2794086564240757
